In [14]:
import pandas as pd

pd.set_option('display.max_colwidth', None)


In [30]:
df = pd.read_csv("./out-min-n-20-max-t-30-clean/accent_data.csv")
df["duration"].min()
# print(df[df["duration"] == df["duration"].max()]["file_path"])

np.float64(16.3525)

In [31]:
df.head()

,file_path,accent,duration,source,split
0,./out-min-n-20-max-t-30-clean/processed_audio/dataset/italian14.wav,italian,25.857063,dataset,train
1,./out-min-n-20-max-t-30-clean/processed_audio/dataset/french36.wav,french,18.962125,dataset,train
2,./out-min-n-20-max-t-30-clean/processed_audio/dataset/spanish69.wav,spanish,20.075187,dataset,train
3,./out-min-n-20-max-t-30-clean/processed_audio/dataset/macedonian24.wav,macedonian,25.400125,dataset,train
4,./out-min-n-20-max-t-30-clean/processed_audio/dataset/german14.wav,german,23.233750,dataset,train


In [32]:
df = df.sort_values(by='file_path').reset_index(drop=True)

In [33]:
df.head()

,file_path,accent,duration,source,split
0,./out-min-n-20-max-t-30-clean/processed_audio/dataset/arabic16.wav,arabic,19.428375,dataset,val
1,./out-min-n-20-max-t-30-clean/processed_audio/dataset/arabic17.wav,arabic,21.505500,dataset,train
2,./out-min-n-20-max-t-30-clean/processed_audio/dataset/arabic2.wav,arabic,27.220875,dataset,train
3,./out-min-n-20-max-t-30-clean/processed_audio/dataset/arabic21.wav,arabic,26.906063,dataset,train
4,./out-min-n-20-max-t-30-clean/processed_audio/dataset/arabic32.wav,arabic,26.891125,dataset,test


In [8]:
percentage = (df[df["duration"] < 30].shape[0] / df.shape[0]) * 100
print(f"Percentage of items with duration below 30: {percentage}%")


Percentage of items with duration below 30: 100.0%


In [29]:
accents = {k:v for (v, k) in enumerate(df["accent"].unique())}
accents

{'korean': 0,
 'arabic': 1,
 'french': 2,
 'english': 3,
 'mandarin': 4,
 'portuguese': 5,
 'turkish': 6,
 'russian': 7,
 'italian': 8,
 'dutch': 9,
 'polish': 10,
 'german': 11,
 'swedish': 12,
 'macedonian': 13,
 'spanish': 14}

In [11]:
import transformers
print(transformers.__version__)
print(transformers.TrainingArguments.__init__.__code__.co_varnames)


4.51.3
('self', 'output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size', 'gradient_accumulation_steps', 'eval_accumulation_steps', 'eval_delay', 'torch_empty_cache_steps', 'learning_rate', 'weight_decay', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'max_grad_norm', 'num_train_epochs', 'max_steps', 'lr_scheduler_type', 'lr_scheduler_kwargs', 'warmup_ratio', 'warmup_steps', 'log_level', 'log_level_replica', 'log_on_each_node', 'logging_dir', 'logging_strategy', 'logging_first_step', 'logging_steps', 'logging_nan_inf_filter', 'save_strategy', 'save_steps', 'save_total_limit', 'save_safetensors', 'save_on_each_node', 'save_only_model', 'restore_callback_states_from_checkpoint', 'no_cuda', 'use_cpu', 'use_mps_device', 'seed', 'data_seed', 'jit_mode_eval', 'use_ipex', 'bf16', 'fp16', 'fp16_opt_level', 'half_precision_ba

In [7]:
from transformers import AutoProcessor, HubertModel
import soundfile as sf

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft")

2025-05-08 12:44:21.064261: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-08 12:44:21.454469: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-05-08 12:44:29.710521: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import HubertModel, HubertConfig, Wav2Vec2FeatureExtractor

# Sample dataset class (replace with your own data loading)
class AccentDataset(Dataset):
    def __init__(self, audio_paths, labels, feature_extractor):
        self.audio_paths = audio_paths
        self.labels = labels
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        # Load audio waveform (e.g., using torchaudio)
        waveform, sr = torchaudio.load(self.audio_paths[idx])
        # Resample if needed, ensure 16kHz
        if sr != 16000:
            waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
        # feature_extractor returns dict with 'input_values' key
        inputs = self.feature_extractor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt")
        labels = torch.tensor(self.labels[idx])
        return inputs.input_values.squeeze(0), labels

# Model wrapper for accent classification
torch.manual_seed(42)

class HubertForAccentClassification(nn.Module):
    def __init__(self, pretrained_model_name="facebook/hubert-base-ls960", num_labels=10, freeze_feature_extractor=True):
        super().__init__()
        # Load pretrained HuBERT
        self.hubert = HubertModel.from_pretrained(pretrained_model_name)
        if freeze_feature_extractor:
            # Freeze the CNN (feature extractor)
            for param in self.hubert.feature_extractor.parameters():
                param.requires_grad = False
        # Encoder output dimension
        hidden_size = self.hubert.config.hidden_size
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size // 2, num_labels)
        )

    def forward(self, input_values, attention_mask=None, labels=None):
        # input_values: (batch, seq_len)
        outputs = self.hubert(input_values, attention_mask=attention_mask)
        # Take mean pooling over time
        hidden_states = outputs.last_hidden_state  # (batch, seq_len, hidden_size)
        pooled = hidden_states.mean(dim=1)        # (batch, hidden_size)
        logits = self.classifier(pooled)          # (batch, num_labels)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        return {"loss": loss, "logits": logits}

# Training loop
def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0.0
    for input_values, labels in dataloader:
        input_values = input_values.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_values, labels=labels)
        loss = outputs["loss"]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

if __name__ == "__main__":
    import torchaudio
    # Replace with your data
    train_audio_paths = ["path/to/audio1.wav", "path/to/audio2.wav", ...]
    train_labels = [0, 1, ...]  # numeric accent labels

    feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, return_attention_mask=False)
    train_dataset = AccentDataset(train_audio_paths, train_labels, feature_extractor)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = HubertForAccentClassification(num_labels=len(set(train_labels))).to(device)
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

    epochs = 5
    for epoch in range(epochs):
        avg_loss = train(model, train_loader, optimizer, device)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

    # Save the fine-tuned model
    model.save_pretrained("./hubert-accent-model")
    print("Model saved.")
